# Week 03

## After-class report

We started our exploration of **Dynamic Programming.** This is covered in [chapter 3](https://jeffe.cs.illinois.edu/teaching/algorithms/book/03-dynprog.pdf) of Jeff Erickson's book.

Additional readings include

- [Bellman's Paper](./OER/Bellman_Theory_of_DP_1954.pdf) from 1954 establishing Dynamic Programming. This is a bit technical but it's a good reference to have if you are interested in the origins of dynamic programming..

- [Introduction to Dynamic Programming](https://cp-algorithms.com/dynamic_programming/intro-to-dp.html) from *Algorithms for Competitive Programming.* This site includes useful pseudocode and links to Leetcode for further practice.


* [Dynamic Programming](https://web.mit.edu/15.053/www/AMP-Chapter-11.pdf) (Chapter 11, from Applied Mathematical Programming, by Bradley, Hax, and Magnanti).

* [Dynamic Programming](https://people.eecs.berkeley.edu/~vazirani/algorithms/chap6.pdf) (Chapter 6 from Algorithms by Dasgupta, Papadimitriou, and Vazirani).'


## String alignment

What is the best way to align the strings `CYCLE` and `BICYCLE`? Typographically, in left-to-right scripts, we tend to align to the left, like so:
```
CYCLE--
BICYCLE
```
The dashes (-) indicates a gap inserted to ensure the proper stretching for alignment.
Computationally, however, we may be interested in maximizing overlap of similar characters, and so the alignment 
```
--CYCLE
BICYCLE
```
may be more preferrable. For strings `CRANE` and `RAIN` we may have more than one alignment:
```
CRANE            CRA-NE
-RAIN            -RAIN-
```
The first option aligns as many letters as possible, while minimizing the use of spacing. The second option maximizes the number of aligned characters at the cost of additional spacing. Which of the two alignments is best? This depends on our priorities. And our priorities can be quantified in a way that determines the optimal alignment of two strings.

Let's assume that every time we use a space we pay a penalty $a_s>0$. Also everytime we have two characters that do not match, we also pay a penalty $a_\text{mismatch} >0$. And finally, when we have two characters matching, we pay no penalty $a_\text{match}\leq 0$. These penalties can be summarized as a function of two characters $x$ and $y$ stacked on top of eachother.

$$
a_{xy} =
\begin{cases}
  >0,\ \text{if}\ x \neq y\ \neq \text{space} \\
  >0,\ \text{if}\ x = \text{space} \neq y \\
  >0,\ \text{if}\ y = \text{space} \neq x  \\
  \leq 0,\ \text{if}\ x = y
\end{cases}
$$

There is no penalty for a space stacked over a space -- such case doesn't exist.

A simple penalty function that favors alignments even at the cost of extra spaces, may look like:

$$
a_{xy} =
\begin{cases}
  5,\ \text{if}\ x \neq y\ \neq \text{space} \\
  1,\ \text{if}\ (x = \text{space} \neq y)\ \text{or}\ (y = \text{space} \neq x )\\
  0,\ \text{if}\ x = y
\end{cases}
$$

For small enough strings, we can find the optimal alignment relatively easy, even with brute force. For two strings with $m$ and $n$ characters respectively, there are 

$$
{\binom{m+n}{n}} = \dfrac{(m+n)!}{m!n!}
$$

possible alignments. If $m=100$ and $m=5$, that's 

$$
\binom{105}{5}=\dfrac{105!}{5!(105-5)!}=96,560,646
$$

and even a laptop can go through them in a few seconds. When the strings get longer, for example $m=10,000$ and $n=500$, the number of alignments becomes astronomical, approximately $1.8\times 10^{871}$ and there is no time to compute so many cases.

## Optimal substructure in string alignment

Consider two strings

\begin{align*}
X_m & = x_1 x_2 \ldots x_m \\
Y_n & = y_1 y_2 \ldots y_n
\end{align*}
whose lengths $m,n$ are not necessarily equal. Their optimal alignment is a pair of strings $\bar{X}$ and $\bar{Y}$ with equal length $\max{(m,n)}\leq L\leq m+n$ such that

$$
{\arg\min} \sum_{\begin{array}{c}\bar x_i\in\bar X\\ \bar y_j\in\bar Y\end{array}} a_{\bar x_i\ \bar y_j}
$$

Typically, we are interested in aligning long strings, such as those representing genetic codes. Their length makes brute-force analysis impossible.

Instead imagine that the optimal alignment $(\bar X, \bar Y)$ is given to us. What would it look like? Both its strings have the same length $L$. We can further split the optimally aligned strings into two parts:

\begin{align*}
\bar X & = X' | \bar x_L \\
\bar Y & = Y' | \bar y_L
\end{align*}

where $X', Y'$ are the first $L-1$ characters of the aligned strings and $\bar x_L$, $\bar y_L$ are the last characters of the aligned strings. Focusing on the last character of $(\bar X, \bar Y)$, we expect one of the following three cases, with respect to the contents of input strings $X_m$ and $Y_n$:

$$
\left|\begin{array}{c}\bar x_L\\ \bar y_L\end{array}\right|=
\begin{cases}
\left|\begin{array}{c}x_m\\ y_n\end{array}\right|: \text{the last character of}\ X_m\ \text{over the last character of}\ Y_n \\ \\
\left|\begin{array}{c}x_m\\ -\end{array}\right|: \text{the last character of}\ X_m\ \text{over a space} \\ \\
\left|\begin{array}{c}-\\ y_n\end{array}\right|: \text{a space over the last character of}\ Y_n
\end{cases}
$$

Each of the cases above reveals important properties for $X'$ and $Y'$, the immediately shorter substrings of the alignment $\bar X$, $\bar Y$.

### Case for $\left|\begin{array}{c}\bar x_L\\ \bar y_L\end{array}\right|=\left|\begin{array}{c}x_m\\ y_n\end{array}\right|$

When the last column of the optimal alignment $(\bar X, \bar Y)$ contains the last characters of the input strings $X_m$ and $Y_n$, then we know that the strings $X'$ and $Y'$ are an optimal alignment of the substrings $X_{m-1}$ and $Y_{n-1}$. For a proof, look at the end of this notebook.

### Case for $\left|\begin{array}{c}\bar x_L\\ \bar y_L\end{array}\right|=\left|\begin{array}{c}x_m\\ -\end{array}\right|$

When the last column of the optimal alignment $(\bar X, \bar Y)$ contains the last character of the input string $X_m$ and a space, then we know that the strings $X'$ and $Y'$ are an optimal alignment of the substrings $X_{m-1}$ and $Y_{n}$. For a proof, look at the end of this notebook.

### Case for $\left|\begin{array}{c}\bar x_L\\ \bar y_L\end{array}\right|=\left|\begin{array}{c}-\\ y_n\end{array}\right|$

When the last column of the optimal alignment $(\bar X, \bar Y)$ contains a space and the last character of the input string $Y_n$, then we know that the strings $X'$ and $Y'$ are an optimal alignment of the substrings $X_{m}$ and $Y_{n-1}$. The proof for this is the same as the case above and described at the end of this notebook.

## Computing penalties

The total penalty for aligning strings $X_m$ and $Y_n$ is measured by some function $P(m,n)$. Based on the analysis above, the alignment of the strings can result from one of the following three cases:
* The alignment of $X_{m-1}$ and $Y_{n-1}$ plus a the penalty associated with $x_m$ aligned with $y_n$; or,
* The alignment of $X_{m-1}$ and $Y_{n}$ plus a the penalty associated with $x_m$ aligned with a space; or,
* The alignment of $X_{m}$ and $Y_{n-1}$ plus a the penalty associated with a space aligned with $y_n$.

Which of these cases prevails depends on the minimum cost associated with it. We can write:

\begin{align*}
P(m,n) = 
    \min( \\ & P(m-1, n-1) + a_{x_m, y_n}, \\
           & P(m-1, n) + a_\text{space}, \\
           & P(m, n-1)+ a_\text{space}\\ )
\end{align*}

Here, $a_\text{space}$ is the penalty $a_{x-}$ or $a_{-y}$. 

Now, of course, we have to compute $P(m-1,n-1)$, $P(m-1,n)$, and $P(m,n-1)$. These penalties can be calculated using the same technique because we are looking at optimal alignments of smaller strings. For example,
\begin{align*}
P(m-1,n-1) = 
    \min( \\ & P(m-2, n-2) + a_{x_{m-1}, y_{n-1}}, \\
           & P(m-2, n-1) + a_\text{space}, \\
           & P(m-1, n-2)+ a_\text{space}\\ )
\end{align*}

In general, we can write


\begin{align*}
P_{i,j} = \min{( \underbrace{P_{i-1,\ j-1} + \alpha_{x_i y_j}}_{\begin{array}{c}\text{\scriptstyle optimal alignment}\\\text{\scriptstyle of}\ \scriptstyle X_{i-1}\ \text{\scriptstyle and}\ \scriptstyle Y_{j-1}\\\text{\scriptstyle with}\ \left|\begin{array}{}\scriptstyle x_i\\\scriptstyle y_j\end{array}\right|\ \text{\scriptstyle added}\\\text{\scriptstyle as last column}\end{array}},\ \
                  \underbrace{P_{i-1,\ j} + \alpha_\text{space}}_{\begin{array}{c}\text{\scriptstyle optimal alignment}\\\text{\scriptstyle of}\ \scriptstyle X_{i-1}\ \text{\scriptstyle and}\ Y_{j}\\\text{\scriptstyle with}\ \left|\begin{array}{c}\scriptstyle x_i\\\text{-}\end{array}\right|\ \text{\scriptstyle added}\\\text{\scriptstyle as last column}\end{array}},\ \
                  \underbrace{P_{i,\ j-1} + \alpha_\text{space}}_{\begin{array}{c}\text{\scriptstyle optimal alignment}\\\text{\scriptstyle of}\ \scriptstyle X_{i}\ \text{\scriptstyle and}\ \scriptstyle Y_{j-1}\\\text{\scriptstyle with}\ \left|\begin{array}{c}\text{-}\\\scriptstyle y_j\end{array}\right|\ \text{\scriptstyle added}\\\text{\scriptstyle as last column}\end{array}})}
\end{align*}
for $0\leq i\leq m$ and $0\leq j\leq n$.

This is a recurrent relation and as such it requires a base case, or initial conditions. We start with the obvious one, $P(0,0)$, the cost of aligning two empty strings. It should be zero, since there is nothing to align. Next we look at the alignment of an empty and a non-empty string: $P(0,j)$ or $P(i,0)$. This cost equals the number of spaces needed to make the empty string equal in length as the non empty string. And so we can write the base cases as:

\begin{align*}
 P(i,0) & = i\times a_\text{space},\ 0\leq i \leq m\\
 P(0,j) & = j\times a_\text{space},\ 0\leq j \leq n
\end{align*}

## Needleman-Wunch scoring

Producing all values $P(i,j)$ is called *Needleman-Wunch scoring* in honor of the two physicians who applied dynamic programming for a biology problem in Evanston, in the 1970s. The values are stored in a $(m+1)\times (n+1)$ array. The initial values of the array are computed from the base cases $P(i,0)$ and $P(0,j)$.

The bottom-right value, $P(m,n)$ is the penalty for the optimal alignment. The alignment itself needs to be computed by tracing back the values in the $P(i,j)$ matrix.


In [ ]:
# Gap penalty
a_gap = 1


# Alignment/misalignment penalty
def a(x, y):
    if x == y:
        return 0
    return 1

In [ ]:
def needleman_wunch(X, Y):
    """Computes minimum alignment score for input strings X and Y"""
    m = len(X)
    n = len(Y)
    # Initialize the penalty table P with m rows and n columns
    P = [[0 for _ in range(n + 1)] for _ in range(m + 1)]
    # Initialize the base cases for P(0,0), P(i,0), and P(0,j)
    for i in range(1, m + 1):
        P[i][0] = i * a_gap
    for j in range(1, n + 1):
        P[0][j] = j * a_gap
    # Fill the rest of the table:
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            P[i][j] = min(
                P[i - 1][j - 1] + a(X[i-1], Y[j-1]),
                P[i - 1][j] + a_gap,
                P[i][j - 1] + a_gap,
            )

    return P

In [ ]:
def fancy_print(P):
    for i in range(len(P)):
        for j in range(len(P[0])):
            print(f"{P[i][j]:5d} ", end="")
        print()

In [ ]:
X = "BICYCLE"
Y = "CYCLE"
P = needleman_wunch(X, Y)
fancy_print(P)


## Tracing back the $P$ matrix

Each value in $P$ is the smallest value of three possible choices described above. Consider this extreme case:
$$
P(i,j) = \min{({\color{CC0000}b},{\color{00CC00}c},{\color{5555FF}d})},\quad {\color{CC0000}b}= {\color{00CC00}c}= {\color{5555FF}d}$$

Each of the three approaches below is feasible but the `min` operation returns only a value, not a direction.

$$
\begin{align*}
{\color{CC0000} P(i-1, j-1)+a_{x_i y_j}} =
{\color{008800} P(i-1, j ) + a_\text{space}} = 
{\color{5555FF} P(i, j-1 ) + a_\text{space}} 
\end{align*}
$$

It is not immediately clear which of the three approaches is the best. Things are easier when 
$
\min{({\color{CC0000}b},{\color{00CC00}c},{\color{5555FF}d})} < \max{({\color{CC0000}b},{\color{00CC00}c},{\color{5555FF}d})}
$.
In this case, only one of the three approaches can be true

$$
\begin{align*}
\min{({\color{CC0000}b},{\color{00CC00}c},{\color{5555FF}d})} = {\color{CC0000}b}\quad & \text{if}\ P(i,j) = P(i-1, j-1)+a_{x_i y_j} \\ 
\min{({\color{CC0000}b},{\color{00CC00}c},{\color{5555FF}d})} = {\color{009900}c}\quad & \text{if}\ P(i,j) =  P(i-1, j ) + a_\text{space} \\
\min{({\color{CC0000}b},{\color{00CC00}c},{\color{5555FF}d})} = {\color{5555FF}d}\quad & \text{if}\ P(i,j) = P(i, j-1 ) + a_\text{space}
\end{align*}
$$

Looking at the bottom-right value $P(m,n)$ we know that it was obtained from any of these three neighboring cells: $\color{CC0000}P(i-1, j-1)$, $\color{008800}P(i-1,j)$ or $\color{5555FF}P(i,j-1)$. Each cell tells a different story.

* $P(m-1,n-1)$ is the penalty for an optimal alignment that ends with $x_{m-1}$ and $y_{n-1}$.
* $P(m-1,n)$ is the penalty for an optimal alignment that ends with $x_{m-1}$ and a space.
* $P(m,n-1)$ is the penalty for an optimal alignment that ends with a space and $y_{n-1}$.

In the example with `CYCLE/BICYCLE`:

* $P(m-1,n-1)$ corresponds to the penalty for the optimal alignment of `CYCL/BICYCL` (whatever that alignment may look). 
* $P(m-1,n)$  corresponds to the penalty for the optimal alignment of `CYCL/BICYCLE` (whatever that alignment may look). 
* $P(m,n-1)$  corresponds to the penalty for the optimal alignment of `CYCLE/BICYCL` (whatever that alignment may look). 

Let assume, for the sake of the argument, that each of the penalties above is the same: say, 10. Each alignemnt above can lead to $P(m,n)$ as follows:

* From $P(m-1,n-1)$, add $x_m$ and $x_n$ to the optimal alignment of `CYCL/BICYCL` (whatever that alignment may look). 
* From $P(m-1,n)$ add $x_m$ and $-$ to the optimal alignment of `CYCL/BICYCLE` (whatever that alignment may look).
* From $P(m,n-1)$ add $-$ and $y_n$ to the optimal alignment of `CYCL/BICYCLE` (whatever that alignment may look).

Each addition, carries a penalty: $0$ for adding $x_m$ and $x_n$ to the optimal alignment of `CYCL/BICYCL` and $1$ for adding a space with either $x_m$ or $y_n$. The minimum operation would have looked at these choices:

$$
 \min{({\color{CC0000}10+0},{\color{00CC00}10+1},{\color{5555FF}10+1})}
$$
and would have assigned

$$
P(m,n) \leftarrow  \min{({\color{CC0000}10+0},{\color{00CC00}10+1},{\color{5555FF}10+1})}
$$


When it's time to traceback $P$, we can look at the value of $P(i,j)$ in general and compare it with its three adjacent cells. Using [cardinal and intercardinal directions](https://en.wikipedia.org/wiki/Points_of_the_compass), those would be the northwest cell, the north cell, and the west cell.

# Your assignment

Put together a program (in the form of an object if you prefer) to align strings using the Needlman-Wunch scoring. Test with
```python
tests = [
    ["CRANE", "RAIN"],
    ["CYCLE", "BICYCLE"],
    ["ASTRONOMY", "GASTRONOMY"],
    ["INTENTION", "EXECUTION"],
    ["AGGTAB", "GXTXAYB"],
    ["GATTACA", "GCATGCU"],
    ["DELICIOUS", "RELIGIOUS"],
]
```

expecting the following results.

```
CRANE -->  |CRA-NE|    matches: 3, mismatches: 0
RAIN  -->  |-RAIN-|    gaps: 3

CYCLE   -->  |--CYCLE|    matches: 5, mismatches: 0
BICYCLE -->  |BICYCLE|    gaps: 2

ASTRONOMY  -->  |-ASTRONOMY|    matches: 9, mismatches: 0
GASTRONOMY -->  |GASTRONOMY|    gaps: 1

INTENTION -->  |INTE-NTION|    matches: 5, mismatches: 3
EXECUTION -->  |-EXECUTION|    gaps: 2

AGGTAB  -->  |AGGT-A-B|    matches: 4, mismatches: 1
GXTXAYB -->  |-GXTXAYB|    gaps: 3

GATTACA -->  |G-ATTACA|    matches: 4, mismatches: 2
GCATGCU -->  |GCA-TGCU|    gaps: 2

DELICIOUS -->  |DELICIOUS|    matches: 7, mismatches: 2
RELIGIOUS -->  |RELIGIOUS|    gaps: 0
```

